In [1]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = 'C:/Users/HP/Desktop/train_test_split/guava/images/train'
test_dir = 'C:/Users/HP/Desktop/train_test_split/guava/images/test'
validation_dir = 'C:/Users/HP/Desktop/train_test_split/guava/images/val'

# Data augmentation for training data
train_datagen = ImageDataGenerator(rescale=1./255)

# Only rescaling for validation and test data
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Load data from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # DenseNet121 expects 224x224 images
    batch_size=32,
    class_mode='binary'  # Binary classification
)

validation_generator = val_test_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False  # Do not shuffle for evaluation
)

# Load DenseNet121 without the top layer
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers (optional for fine-tuning)
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global average pooling
x = Dense(1024, activation='relu')(x)  # Fully connected layer
predictions = Dense(1, activation='sigmoid')(x)  # Output layer for binary classification

# Define the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=20
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')



Found 2224 images belonging to 2 classes.
Found 478 images belonging to 2 classes.
Found 479 images belonging to 2 classes.


C:\Users\HP\Documents\anaconda2\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
69/69 ━━━━━━━━━━━━━━━━━━━━ 173s 2s/step - accuracy: 0.8076 - loss: 0.4473 - val_accuracy: 0.9375 - val_loss: 0.2117
Epoch 2/20
 1/69 ━━━━━━━━━━━━━━━━━━━━ 1:38 1s/step - accuracy: 0.9688 - loss: 0.1410

C:\Users\HP\Documents\anaconda2\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


69/69 ━━━━━━━━━━━━━━━━━━━━ 22s 305ms/step - accuracy: 0.9688 - loss: 0.1410 - val_accuracy: 0.9263 - val_loss: 0.2127
Epoch 3/20
69/69 ━━━━━━━━━━━━━━━━━━━━ 206s 3s/step - accuracy: 0.9525 - loss: 0.1703 - val_accuracy: 0.9487 - val_loss: 0.1551
Epoch 4/20
69/69 ━━━━━━━━━━━━━━━━━━━━ 26s 363ms/step - accuracy: 0.9062 - loss: 0.2187 - val_accuracy: 0.9420 - val_loss: 0.1599
Epoch 5/20
69/69 ━━━━━━━━━━━━━━━━━━━━ 228s 3s/step - accuracy: 0.9563 - loss: 0.1320 - val_accuracy: 0.9643 - val_loss: 0.1248
Epoch 6/20
69/69 ━━━━━━━━━━━━━━━━━━━━ 35s 479ms/step - accuracy: 0.9375 - loss: 0.0796 - val_accuracy: 0.9576 - val_loss: 0.1308
Epoch 7/20
69/69 ━━━━━━━━━━━━━━━━━━━━ 202s 3s/step - accuracy: 0.9709 - loss: 0.0941 - val_accuracy: 0.9665 - val_loss: 0.1041
Epoch 8/20
69/69 ━━━━━━━━━━━━━━━━━━━━ 41s 570ms/step - accuracy: 0.9688 - loss: 0.0794 - val_accuracy: 0.9732 - val_loss: 0.1047
Epoch 9/20
69/69 ━━━━━━━━━━━━━━━━━━━━ 225s 3s/step - accuracy: 0.9769 - loss: 0.0842 - val_accuracy: 0.9531 - val_